#Model Training

In [ ]:
import os
import cv2
import math
import random
import numpy as np
import datetime as dt
import tensorflow as tf
from collections import deque
import matplotlib.pyplot as plt
from google.colab import drive
from moviepy.editor import *
from IPython.display import Image
%matplotlib inline
 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Error while fetching file: The read operation timed out.
Try 2. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)3530752/45929032 bytes (7.7%)7151616/45929032 bytes (15.6%)10780672/45929032 bytes (23.5%)14401536/45929032 bytes (31.4%)17981440/45929032 bytes (39.2%)21618688/45929032 bytes (47.1%)25182208/45929032 bytes (54.8%)28737536/45929032 bytes (62.6%)32342016/45929032 bytes (70.4%)35921920/45929032 bytes (78.2%)39395328/45929032 bytes (85.8%)42

In [ ]:
drive.mount('/content/drive')
# Specify the height and width to which each video frame will be resized in our dataset.
IMAGE_HEIGHT , IMAGE_WIDTH = 64, 64
# Specify the number of frames of a video that will be fed to the model as one sequence.
SEQUENCE_LENGTH = 20
seed_constant = 786
np.random.seed(seed_constant)
random.seed(seed_constant)
tf.random.set_seed(seed_constant)
# Reading names
# <=========================================ADD YOUR DRIVE CLIPPED FOLDERS PATH IN HER ========================================>
videos = {'welding':os.listdir('drive/MyDrive/Colab Notebooks/welding videos/clipped'),'spray painting':os.listdir('drive/MyDrive/Colab Notebooks/spray painting videos/clipped'),'using a wrench':os.listdir('drive/MyDrive/Colab Notebooks/using a wrench videos/clipped')}

Mounted at /content/drive


In [ ]:
def frames_extraction(video_path):
    '''
    This function will extract the required frames from a video after resizing and normalizing them.
    Args:
        video_path: The path of the video in the disk, whose frames are to be extracted.
    Returns:
        frames_list: A list containing the resized and normalized frames of the video.
    '''
    # Declare a list to store video frames.
    frames_list = []
    # Read the Video File using the VideoCapture object.
    video_reader = cv2.VideoCapture(video_path)
    # Get the total number of frames in the video.
    video_frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))
    # Calculate the the interval after which frames will be added to the list.
    skip_frames_window = max(int(video_frames_count/SEQUENCE_LENGTH), 1)
    # Iterate through the Video Frames.
    for frame_counter in range(SEQUENCE_LENGTH):
 
        # Set the current frame position of the video.
        video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * skip_frames_window)
 
        # Reading the frame from the video. 
        success, frame = video_reader.read() 
 
        # Check if Video frame is not successfully read then break the loop
        if not success:
            break
 
        # Resize the Frame to fixed height and width.
        resized_frame = cv2.resize(frame, (IMAGE_HEIGHT, IMAGE_WIDTH))
        
        # Normalize the resized frame by dividing it with 255 so that each pixel value then lies between 0 and 1
        normalized_frame = resized_frame / 255
        
        # Append the normalized frame into the frames list
        frames_list.append(normalized_frame)
    
    # Release the VideoCapture object. 
    video_reader.release()
 
    # Return the frames list.
    return frames_list

In [ ]:
def create_dataset():
    '''
    This function will extract the data of the selected classes and create the required dataset.
    Returns:
        features:          A list containing the extracted frames of the videos.
        labels:            A list containing the indexes of the classes associated with the videos.
        video_files_paths: A list containing the paths of the videos in the disk.
    '''

    # Declared Empty Lists to store the features, labels and video file path values.
    features = []
    labels = []
    video_files_paths = []
    
    # Iterating through all the classes mentioned in the classes list
    for classname,file_names in videos.items():
      print(f'Extracting {classname} Frames')
      for video in file_names:       
        # Get the complete video path.
        video_file_path = f'drive/MyDrive/Colab Notebooks/{classname} videos/clipped/{video}'
        print(f'processing {video_file_path}')
        # Extract the frames of the video file.
        frames = frames_extraction(video_file_path)
        # Check if the extracted frames are equal to the SEQUENCE_LENGTH specified above.
        # So ignore the vides having frames less than the SEQUENCE_LENGTH.
        if len(frames) == SEQUENCE_LENGTH:
            # Append the data to their repective lists.
            features.append(frames)
            labels.append( classname)
            video_files_paths.append(video_file_path)

    # Converting the list to numpy arrays
    features = np.asarray(features)
    labels = np.array(labels)  
    
    # Return the frames, class index, and video file path.
    return features, labels, video_files_paths

In [ ]:
# Create the dataset.
features, labels, video_files_paths = create_dataset()

Extracting welding Frames
processing drive/MyDrive/Colab Notebooks/welding videos/clipped/Master Welder William.mp4
processing drive/MyDrive/Colab Notebooks/welding videos/clipped/Eastwood MIG Welder 250 Amp - Welding Sheet Metal to 12 Steel Plate!.mp4
processing drive/MyDrive/Colab Notebooks/welding videos/clipped/Welding cast iron.mp4
processing drive/MyDrive/Colab Notebooks/welding videos/clipped/Steel Mig Welding.mp4
processing drive/MyDrive/Colab Notebooks/welding videos/clipped/Dumpster Refurbishing - Welding New Nose Roller on 40-Yard Roll-Off.mp4
processing drive/MyDrive/Colab Notebooks/welding videos/clipped/Manufacturing and welding process.mp4
processing drive/MyDrive/Colab Notebooks/welding videos/clipped/Welding Close Up.mp4
processing drive/MyDrive/Colab Notebooks/welding videos/clipped/MIG Welding Mig Welding demo.mp4
processing drive/MyDrive/Colab Notebooks/welding videos/clipped/Welding a Tree Brace - Prune Like a Pro.mp4
processing drive/MyDrive/Colab Notebooks/weldin

In [ ]:
# Saving the dataset in drive for later use 
np.save('drive/MyDrive/Colab Notebooks/saved data/features', features)
np.save('drive/MyDrive/Colab Notebooks/saved data/labels', labels)
np.save('drive/MyDrive/Colab Notebooks/saved data/paths', video_files_paths)

In [ ]:
# To load data from the saved np files
features = np.load('drive/MyDrive/Colab Notebooks/saved data/features.npy')
labels = np.load('drive/MyDrive/Colab Notebooks/saved data/labels.npy')
video_files_paths = np.load('drive/MyDrive/Colab Notebooks/saved data/paths.npy')

In [ ]:
# Label encode the labels
le = LabelEncoder()
labels = le.fit_transform(labels)

In [ ]:
# Using Keras's to_categorical method to convert labels into one-hot-encoded vectors
one_hot_encoded_labels = to_categorical(labels)
# Split the Data into Train ( 80% ) and Test Set ( 20% ).
features_train, features_test, labels_train, labels_test = train_test_split(features, one_hot_encoded_labels, test_size = 0.20, shuffle = True, random_state = seed_constant)

In [ ]:
def create_LRCN_model():
    '''
    This function will construct the required LRCN model.
    Returns:
        model: It is the required constructed LRCN model.
    '''
 
    # We will use a Sequential model for model construction.
    model = Sequential()
    
    # Define the Model Architecture.
    ########################################################################################################################
    
    model.add(TimeDistributed(Conv2D(16, (3, 3), padding='same',activation = 'relu'),
                              input_shape = (SEQUENCE_LENGTH, IMAGE_HEIGHT, IMAGE_WIDTH, 3)))
    
    model.add(TimeDistributed(MaxPooling2D((4, 4)))) 
    model.add(TimeDistributed(Dropout(0.25)))
    
    model.add(TimeDistributed(Conv2D(32, (3, 3), padding='same',activation = 'relu')))
    model.add(TimeDistributed(MaxPooling2D((4, 4))))
    model.add(TimeDistributed(Dropout(0.25)))
    
    model.add(TimeDistributed(Conv2D(64, (3, 3), padding='same',activation = 'relu')))
    model.add(TimeDistributed(MaxPooling2D((2, 2))))
    model.add(TimeDistributed(Dropout(0.25)))
    
    model.add(TimeDistributed(Conv2D(64, (3, 3), padding='same',activation = 'relu')))
    model.add(TimeDistributed(MaxPooling2D((2, 2))))
    #model.add(TimeDistributed(Dropout(0.25)))
                                      
    model.add(TimeDistributed(Flatten()))
                                      
    model.add(LSTM(32))
                                      
    model.add(Dense(len(videos), activation = 'softmax'))
 
    ########################################################################################################################
 
    # Display the models summary.
    model.summary()
    
    # Return the constructed LRCN model.
    return model

In [ ]:
LRCN_model = create_LRCN_model()
 
# Display the success message.
print("Model Created Successfully!")

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed (TimeDistr  (None, 20, 64, 64, 16)   448       
 ibuted)                                                         
                                                                 
 time_distributed_1 (TimeDis  (None, 20, 16, 16, 16)   0         
 tributed)                                                       
                                                                 
 time_distributed_2 (TimeDis  (None, 20, 16, 16, 16)   0         
 tributed)                                                       
                                                                 
 time_distributed_3 (TimeDis  (None, 20, 16, 16, 32)   4640      
 tributed)                                                       
                                                                 
 time_distributed_4 (TimeDis  (None, 20, 4, 4, 32)     0

In [ ]:
# Create an Instance of Early Stopping Callback.
early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 10, mode = 'min', restore_best_weights = True)
 
# Compile the model and specify loss function, optimizer and metrics to the model.
LRCN_model.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ["accuracy"])

# Start training the model.
LRCN_model_training_history = LRCN_model.fit(x = features_train, y = labels_train, epochs = 100, batch_size = 4 , shuffle = True, validation_split = 0.2, callbacks = [early_stopping_callback])

Epoch 1/100
324/324 [==============================] - 36s 103ms/step - loss: 0.9766 - accuracy: 0.4880 - val_loss: 1.0007 - val_accuracy: 0.4815
Epoch 2/100
324/324 [==============================] - 34s 104ms/step - loss: 0.9213 - accuracy: 0.5213 - val_loss: 0.9746 - val_accuracy: 0.4877
Epoch 3/100
324/324 [==============================] - 33s 101ms/step - loss: 0.8646 - accuracy: 0.5669 - val_loss: 0.9850 - val_accuracy: 0.5216
Epoch 4/100
324/324 [==============================] - 33s 103ms/step - loss: 0.8166 - accuracy: 0.5978 - val_loss: 1.0492 - val_accuracy: 0.4383
Epoch 5/100
324/324 [==============================] - 33s 102ms/step - loss: 0.7956 - accuracy: 0.6326 - val_loss: 0.9247 - val_accuracy: 0.5401
Epoch 6/100
324/324 [==============================] - 33s 101ms/step - loss: 0.7612 - accuracy: 0.6589 - val_loss: 0.9048 - val_accuracy: 0.5463
Epoch 7/100
324/324 [==============================] - 33s 100ms/step - loss: 0.7342 - accuracy: 0.6558 - val_loss: 0.8377 -

In [ ]:
# To test the model 
model_evaluation_history = LRCN_model.evaluate(features_test, labels_test)

13/13 [==============================] - 3s 192ms/step - loss: 0.7818 - accuracy: 0.6667
